### 0. Imports

In [43]:
%load_ext autoreload
%autoreload 2

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualizaciones
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt

# Vigilar progreso bucles
# -----------------------------------------------------------------------
from tqdm import tqdm

# Gestionar los warnings
# -----------------------------------------------------------------------
import warnings

# modificar el path
# -----------------------------------------------------------------------
import sys
sys.path.append("..")

# importar funciones de soporte
# -----------------------------------------------------------------------
import src.soporte_eda as se
import src.soporte_preprocesamiento as sp

# evaluar objetos literales
# -----------------------------------------------------------------------
from ast import literal_eval 

# statistics functions
# -----------------------------------------------------------------------
from scipy.stats import pearsonr, spearmanr, pointbiserialr


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Introduction - Employee Retention

# 2. Preliminary analysis and data cleaning

## 2.1 Import data

We import the main dataframe first.

In [44]:
general_data_df = pd.read_csv("../data/general_data.csv")
general_data_df.columns = [col.lower() for col in general_data_df.columns]
general_data_df.head(2)

,age,attrition,businesstravel,department,distancefromhome,education,educationfield,employeecount,employeeid,gender,...,numcompaniesworked,over18,percentsalaryhike,standardhours,stockoptionlevel,totalworkingyears,trainingtimeslastyear,yearsatcompany,yearssincelastpromotion,yearswithcurrmanager
0,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,1,Female,...,1.0,Y,11,8,0,1.0,6,1,0,0
1,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,1,2,Female,...,0.0,Y,23,8,1,6.0,3,5,1,4


Then, the secondary ones. 

The emplyee survey:

In [45]:
employee_survey_df = pd.read_csv("../data/employee_survey_data.csv")
employee_survey_df.columns = [col.lower() for col in employee_survey_df.columns]
employee_survey_df.head(2)

,employeeid,environmentsatisfaction,jobsatisfaction,worklifebalance
0,1,3.0,4.0,2.0
1,2,3.0,2.0,4.0


And the manager survey.

In [46]:
manager_survey_df = pd.read_csv("../data/manager_survey_data.csv")
manager_survey_df.columns = [col.lower() for col in manager_survey_df.columns]
manager_survey_df.head(2)

,employeeid,jobinvolvement,performancerating
0,1,3,3
1,2,2,4


## 2.2 Explore dataframe

### 2.2.1 General_data

Let's perform a preliminary exploration to know what cleaning might be necessary.

In [47]:
se.exploracion_dataframe(general_data_df)

El número de datos es 4410 y el de columnas es 24

 ..................... 

Las primeras filas del dataframe son:


,age,attrition,businesstravel,department,distancefromhome,education,educationfield,employeecount,employeeid,gender,...,numcompaniesworked,over18,percentsalaryhike,standardhours,stockoptionlevel,totalworkingyears,trainingtimeslastyear,yearsatcompany,yearssincelastpromotion,yearswithcurrmanager
0,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,1,Female,...,1.0,Y,11,8,0,1.0,6,1,0,0
1,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,1,2,Female,...,0.0,Y,23,8,1,6.0,3,5,1,4



 ..................... 

Los tipos de las columnas y sus valores únicos son:


,tipo_dato,conteo
age,int64,43
attrition,object,2
businesstravel,object,3
department,object,3
distancefromhome,int64,29
education,int64,5
educationfield,object,6
employeecount,int64,1
employeeid,int64,4410
gender,object,2



 ..................... 

Los duplicados que tenemos en el conjunto de datos son: 0

 ..................... 

Los nulos que tenemos en el conjunto de datos son:


...

,count,pct
percentsalaryhike,,
11,630,14.3
13,627,14.2
14,603,13.7
12,594,13.5
15,303,6.9


La columna STOCKOPTIONLEVEL tiene 4 valores únicos, de los cuales los primeros son:


,count,pct
stockoptionlevel,,
0,1893,42.9
1,1788,40.5
2,474,10.7
3,255,5.8


La columna TRAININGTIMESLASTYEAR tiene 7 valores únicos, de los cuales los primeros son:


,count,pct
trainingtimeslastyear,,
2,1641,37.2
3,1473,33.4
4,369,8.4
5,357,8.1
1,213,4.8





**Duplicates**
- There are **0** row level duplicates in the dataset.

Inspecting if there are duplicates by id:


In [48]:
general_data_df.duplicated("employeeid").sum()

0

**Missing values**

A negligible number of missing values in the columns:
- numcompaniesworked. 	0.430839%
- totalworkingyears.	0.204082%

These will be imputed inside the pipeline via a simple imputer, at first, given their low impact.

**Low variability columns**

- After the exploration script dropped unique value columns, no other column has low variability of categories or values.


### Categorical columns values

No major problems are observed from either high cardinality, data type errors or typos.


### Numerical columns

For numerical columns, the quickest way to assess value ranges and other statistics is through a descriptive summary:

In [49]:
display(general_data_df.describe().T)
general_data_df.select_dtypes(np.number).nunique().reset_index()

,count,mean,std,min,25%,50%,75%,max
age,4410.0,36.923810,9.133301,18.0,30.00,36.0,43.00,60.0
distancefromhome,4410.0,9.192517,8.105026,1.0,2.00,7.0,14.00,29.0
employeeid,4410.0,2205.500000,1273.201673,1.0,1103.25,2205.5,3307.75,4410.0
monthlyincome,4410.0,65029.312925,47068.888559,10090.0,29110.00,49190.0,83800.00,199990.0
totalworkingyears,4401.0,11.279936,7.782222,0.0,6.00,10.0,15.00,40.0
yearsatcompany,4410.0,7.008163,6.125135,0.0,3.00,5.0,9.00,40.0
yearssincelastpromotion,4410.0,2.187755,3.221699,0.0,0.00,1.0,3.00,15.0
yearswithcurrmanager,4410.0,4.123129,3.567327,0.0,2.00,3.0,7.00,17.0


,index,0
0,age,43
1,distancefromhome,29
2,employeeid,4410
3,monthlyincome,1349
4,totalworkingyears,40
5,yearsatcompany,37
6,yearssincelastpromotion,16
7,yearswithcurrmanager,18


No presence of outliers is detected judging from the ranges of features.

Yearswithcurrmanager and yearssincelastpromotion could also be treated as object.


In [50]:
general_data_df[["yearssincelastpromotion","yearswithcurrmanager"]] = general_data_df[["yearssincelastpromotion","yearswithcurrmanager"]].astype("object")

The employeeid should normally be dropped but must be kept to join the tables.

### 2.2.2 Employee_survey

In [51]:
se.exploracion_dataframe(employee_survey_df)

El número de datos es 4410 y el de columnas es 4

 ..................... 

Las primeras filas del dataframe son:


,employeeid,environmentsatisfaction,jobsatisfaction,worklifebalance
0,1,3.0,4.0,2.0
1,2,3.0,2.0,4.0



 ..................... 

Los tipos de las columnas y sus valores únicos son:


,tipo_dato,conteo
employeeid,int64,4410
environmentsatisfaction,float64,4
jobsatisfaction,float64,4
worklifebalance,float64,4



 ..................... 

Los duplicados que tenemos en el conjunto de datos son: 0

 ..................... 

Los nulos que tenemos en el conjunto de datos son:


...

,count,pct
environmentsatisfaction,,
3.0,1350,30.6
4.0,1334,30.2
2.0,856,19.4
1.0,845,19.2


La columna JOBSATISFACTION tiene 4 valores únicos, de los cuales los primeros son:


,count,pct
jobsatisfaction,,
4.0,1367,31.0
3.0,1323,30.0
1.0,860,19.5
2.0,840,19.0


La columna WORKLIFEBALANCE tiene 4 valores únicos, de los cuales los primeros son:


,count,pct
worklifebalance,,
3.0,2660,60.3
2.0,1019,23.1
4.0,454,10.3
1.0,239,5.4


### 2.2.3 Manager_survey

In [52]:
se.exploracion_dataframe(manager_survey_df)

El número de datos es 4410 y el de columnas es 3

 ..................... 

Las primeras filas del dataframe son:


,employeeid,jobinvolvement,performancerating
0,1,3,3
1,2,2,4



 ..................... 

Los tipos de las columnas y sus valores únicos son:


,tipo_dato,conteo
employeeid,int64,4410
jobinvolvement,int64,4
performancerating,int64,2



 ..................... 

Los duplicados que tenemos en el conjunto de datos son: 0

 ..................... 

Los nulos que tenemos en el conjunto de datos son:


,%_nulos



 ..................... 

Comprobamos una representación mínima para valores numéricos:

 ..................... 

Comprobamos una representación mínima para valores numéricos:
● La variable jobinvolvement tiene 4 < 15 valores únicos. Se convierte a objeto.
● La variable performancerating tiene 2 < 15 valores únicos. Se convierte a objeto.

 ..................... 

Los valores que tenemos para las columnas categóricas son: 
La columna JOBINVOLVEMENT tiene 4 valores únicos, de los cuales los primeros son:


,count,pct
jobinvolvement,,
3,2604,59.0
2,1125,25.5
4,432,9.8
1,249,5.6


La columna PERFORMANCERATING tiene 2 valores únicos, de los cuales los primeros son:


,count,pct
performancerating,,
3,3732,84.6
4,678,15.4


# 3. Join dataframes

Merge to join the three datasats into one.

In [ ]:
employee_attrition = general_data_df.merge(employee_survey_df, how="inner").merge(manager_survey_df, how="inner")

Drop the employee_id column.

In [56]:
employee_attrition.drop(columns="employeeid", inplace=True)

# 4. Export

Export the cleaned data in parquet format to retain data types and compress the information. This data will be used in the next phase of the project `notebooks\2_EDA.ipynb`.

In [58]:
employee_attrition.to_parquet("../data/cleaned/employee_attrition_clean.parquet")
employee_attrition.to_pickle("../data/cleaned/employee_attrition_clean.pkl")